<a href="https://colab.research.google.com/github/TarteTatin1212/contents_search/blob/main/contents_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/MyDrive/Senior_Project'

In [ ]:
!pip install --upgrade google-api-python-client
!pip install pytz

In [ ]:
from apiclient.discovery import build
from datetime import datetime, timedelta
import pytz

In [ ]:
# APIキーを設定
api_key = ""

# APIクライアントを構築
youtube = build("youtube", "v3", developerKey=api_key)

# 日本時間の1時間前をISO 8601形式の文字列に変換
one_hour_ago = (datetime.now(pytz.timezone("Asia/Tokyo")) - timedelta(hours=1)).isoformat()


In [ ]:
# 検索リクエストを実行
search_response = youtube.search().list(
    q="",  # 検索クエリ（ここでは全ての動画を対象）
    part="id",  # 動画のIDを取得
    type="video",  # ビデオのみを対象
    publishedAfter=one_hour_ago,  # 1時間前以降に公開された動画のみを対象
    maxResults=50,  # 最大50件の結果を取得
    regionCode="JP",  # 日本で公開された動画のみを対象
).execute()

for item in search_response["items"]:
    video_id = item["id"]["videoId"]

    # 動画の詳細情報を取得
    video_response = youtube.videos().list(
        part="snippet,statistics",  # 動画の基本情報と視聴回数を取得
        id=video_id,  # 検索で取得した動画IDを指定
    ).execute()

    for video in video_response["items"]:
        # チャンネル名、動画のサムネイル、再生回数を表示
        print(f"Channel: {video['snippet']['channelTitle']}")
        print(f"Thumbnail: {video['snippet']['thumbnails']['default']['url']}")
        print(f"View count: {video['statistics']['viewCount']}")

In [ ]:
import json
import pandas as pd

In [ ]:
# データを保存するリスト
data = []

# 動画ごとに必要な情報を取得
for item in search_response["items"]:
    video_id = item["id"]["videoId"]

    # 動画の詳細情報を取得
    video_response = youtube.videos().list(
        part="snippet,statistics",  # 動画の基本情報と視聴回数を取得
        id=video_id,  # 検索で取得した動画IDを指定
    ).execute()

    for item in video_response["items"]:
        channel_title = item["snippet"]["channelTitle"]  # チャンネル名
        thumbnail_url = item["snippet"]["thumbnails"]["default"]["url"]  # サムネイルURL
        view_count = item["statistics"]["viewCount"]  # 再生回数

        # 取得した情報を辞書に格納
        data.append({
            "channelTitle": channel_title,
            "thumbnailUrl": thumbnail_url,
            "viewCount": view_count
        })

# JSON形式で保存
with open('youtube_data.json', 'w') as json_file:
    json.dump(data, json_file)

# CSV形式で保存
df = pd.DataFrame(data)
df.to_csv('youtube_data.csv', index=False)